# EBSI Wallet Conformance (Natural Person)

> EBSI v2 DID for Natural Person(s) is introduced. 

## Setup conformance headers

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

headers = {
    "Conformance": str(uuid.uuid4()),
}

## Setup wallet

In [2]:
from ebsi_wallet.main import wallet

# Setup wallet
client = await wallet("init_v2")


## ISSUE_011

In [3]:
from ebsi_wallet.main import conformance
from ebsi_wallet.util import parse_query_string_parameters_from_url

result = await conformance("issuerInitiate", headers=headers)

console.log("issuerInitiate", result)

session_token = result.replace("openid://", "")
credential_type = parse_query_string_parameters_from_url(
    session_token).get("credential_type")[0]
issuer_url = parse_query_string_parameters_from_url(
    session_token).get("issuer")[0]

console.log("credential_type", credential_type)

options = {
    "client": client,
    "credential_type" : credential_type
}

result = await conformance("issuerAuthorize", options=options)
console.log("issuerAuthorize", result)


[14:02:25] issuerInitiate                                                                           ]8;id=285625;file:///tmp/ipykernel_14447/4201365539.py\4201365539.py]8;;\:]8;id=331418;file:///tmp/ipykernel_14447/4201365539.py#6\6]8;;\
           openid://initiate_issuance?issuer=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2F                
           v2&credential_type=https%3A%2F%2Fapi-conformance.ebsi.eu%2Ftrusted-schemas-registry%2Fv2                
           %2Fschemas%2Fz22ZAMdQtNLwi51T2vdZXGGZaYyjrsuP1yzWyXZirCAHv&conformance=92c4c1d3-ce28-49b                
           f-908e-7cff738a6b16                                                                                     

           credential_type                                                                         ]8;id=642340;file:///tmp/ipykernel_14447/4201365539.py\4201365539.py]8;;\:]8;id=564897;file:///tmp/ipykernel_14447/4201365539.py#14\14]8;;\
           https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z22ZAMdQtNLwi51T2vd                 
           ZXGGZaYyjrsuP1yzWyXZirCAHv                                                                              

           issuerAuthorize                                                                         ]8;id=522013;file:///tmp/ipykernel_14447/4201365539.py\4201365539.py]8;;\:]8;id=562093;file:///tmp/ipykernel_14447/4201365539.py#22\22]8;;\
           {'state': 'ae0542facf31', 'code': '1424cdc0f1f749305ceb'}                                               

## ISSUE_021

In [4]:
options = {
    "did_version": client.did_version,
    "code": result["code"]
}

result = await conformance("issuerToken", options=options, headers=headers)
console.log("issuerToken", result)


[14:02:26] issuerToken                                                                              ]8;id=894189;file:///tmp/ipykernel_14447/1256846934.py\1256846934.py]8;;\:]8;id=651276;file:///tmp/ipykernel_14447/1256846934.py#7\7]8;;\
           {                                                                                                       
               'token_type': 'Bearer',                                                                             
               'access_token': '49b8190f9a964bc042dc',                                                             
               'expires_in': 3600,                                                                                 
               'c_nonce': '4ea0aecece81536781e7',                                                                  
               'id_token':                                                                                         
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpYXQiOjE2NzMzMzk1NDYsInN1YiI6Imh0dHBzOi8vbG9j                
           YWxob3N0OjMwMDAiLCJhdWQiOiJodHRwczovL2xvY2FsaG9zdDozMDAwIiwiaXNzIjoiZGlkOmVic2k6emNmY3dH                
           akxCb2pjenc5eWhtVUZFM1oifQ.3BKeBgqcFaFkDzAM1qpn1Fj9zOXTKnFuQbSWw3KnHLBLUYIKbGZZ1Bfv4bNnu                
           4F_ugfBGzc6b8LDXU56CN4iEA'                                                                              
           }                                                                                                       

## ISSUE_031

In [5]:
from ebsi_wallet.did_jwt import decode_jwt

options = {
    "c_nonce": result["c_nonce"],
    "client": client,
    "issuer_url": issuer_url,
    "credential_type": credential_type,
}

access_token = result["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}",
    "Conformance": headers["Conformance"]
}

result = await conformance("issuerCredential", options=options, headers=headers)
console.log("issuerCredential", result)

issue_031_credential = result["credential"]

credential = decode_jwt(result["credential"])

console.log("Received credential", credential)

           issuerCredential                                                                         ]8;id=520510;file:///tmp/ipykernel_14447/608560569.py\608560569.py]8;;\:]8;id=249009;file:///tmp/ipykernel_14447/608560569.py#18\18]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NksiLCJ0eXAiOiJKV1QiLCJraWQiOiJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkU                
           zWiNrZXlzLTEifQ.eyJqdGkiOiJ1cm46ZGlkOmFhOGMxYWVkLTlmNGUtNGE0OS04NWZlLWUxYjY0NGZkMDgxYiIs                
           InN1YiI6ImRpZDplYnNpOnprTmtWYlVoN29wMmVOMkhHbUZrTUN2aVRldVN1U2t5Z0tUWVBNS1NFdGFuQyIsImlz                
           cyI6ImRpZDplYnNpOnpjZmN3R2pMQm9qY3p3OXlobVVGRTNaIiwibmJmIjoxNjczMzM5NTQ2LCJpYXQiOjE2NzMz                
           Mzk1NDYsInZjIjp7IkBjb250ZXh0IjpbImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0s                
           ImlkIjoidXJuOmRpZDphYThjMWFlZC05ZjRlLTRhNDktODVmZS1lMWI2NDRmZDA4MWIiLCJ0eXBlIjpbIlZlcmlm                
           aWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiVmVyaWZpYWJsZUlkIl0sImlzc3VlciI6                
           ImRpZDplYnNpOnpjZmN3R2pMQm9qY3p3OXlobVVGRTNaIiwiaXNzdWFuY2VEYXRlIjoiMjAyMy0wMS0xMFQwODoz                
           MjoyNloiLCJ2YWxpZEZyb20iOiIyMDIzLTAxLTEwVDA4OjMyOjI2WiIsImlzc3VlZCI6IjIwMjMtMDEtMTBUMDg6                
           MzI6MjZaIiwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJkaWQ6ZWJzaTp6a05rVmJVaDdvcDJlTjJIR21Ga01D                
           dmlUZXVTdVNreWdLVFlQTUtTRXRhbkMiLCJwZXJzb25hbElkZW50aWZpZXIiOiJJVC9ERS8xMjM0IiwiZmFtaWx5                
           TmFtZSI6IkNhc3RhZmlvcmkiLCJmaXJzdE5hbWUiOiJCaWFuY2EiLCJkYXRlT2ZCaXJ0aCI6IjE5MzAtMTAtMDEi                
           fSwiY3JlZGVudGlhbFNjaGVtYSI6eyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvdHJ1c3Rl                
           ZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejIyWkFNZFF0Tkx3aTUxVDJ2ZFpYR0daYVl5anJzdVAxeXpX                
           eVhaaXJDQUh2IiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9fX0.jkp4PY8lQDW2l8Pcwgum                
           sXkdd1ujUTot7g3f6vk1C0kZtZu56kESg2W0OL_IjMc2Xi_Yrh1WAiHSdoLW-LoaCQ',                                    
               'c_nonce': '8bab13beb42f6e244689',                                                                  
               'c_nonce_expires_in': 900                                                                           
           }                                                                                                       

           Received credential                                                                      ]8;id=483400;file:///tmp/ipykernel_14447/608560569.py\608560569.py]8;;\:]8;id=17867;file:///tmp/ipykernel_14447/608560569.py#24\24]8;;\
           {                                                                                                       
               'header': {                                                                                         
                   'alg': 'ES256K',                                                                                
                   'typ': 'JWT',                                                                                   
                   'kid': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z#keys-1'                                                
               },                                                                                                  
               'payload': {                                                                                        
                   'jti': 'urn:did:aa8c1aed-9f4e-4a49-85fe-e1b644fd081b',                                          
                   'sub': 'did:ebsi:zkNkVbUh7op2eN2HGmFkMCviTeuSuSkygKTYPMKSEtanC',                                
                   'iss': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                                      
                   'nbf': 1673339546,                                                                              
                   'iat': 1673339546,                                                                              
                   'vc': {                                                                                         
                       '@context': ['https://www.w3.org/2018/credentials/v1'],                                     
                       'id': 'urn:did:aa8c1aed-9f4e-4a49-85fe-e1b644fd081b',                                       
                       'type': ['VerifiableCredential', 'VerifiableAttestation', 'VerifiableId'],                  
                       'issuer': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                               
                       'issuanceDate': '2023-01-10T08:32:26Z',                                                     
                       'validFrom': '2023-01-10T08:32:26Z',                                                        
                       'issued': '2023-01-10T08:32:26Z',                                                           
                       'credentialSubject': {                                                                      
                           'id': 'did:ebsi:zkNkVbUh7op2eN2HGmFkMCviTeuSuSkygKTYPMKSEtanC',                         
                           'personalIdentifier': 'IT/DE/1234',                                                     
                           'familyName': 'Castafiori',                                                             
                           'firstName': 'Bianca',                                                                  
                           'dateOfBirth': '1930-10-01'                                                             
                       },                                                                                          
                       'credentialSchema': {                                                                       
                           'id':                                                                                   
           'https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z22ZAMdQtNLwi51T2vd                
           ZXGGZaYyjrsuP1yzWyXZirCAHv',                                                                            
                           'type': 'FullJsonSchemaValidator2021'                                                   
                       }                    

## VERIFY_011

In [6]:
headers = {
    "Conformance": headers.get("Conformance")
}
options = {
    "did_version": "v2"
}

result = await conformance("verifierAuthRequest", options=options, headers=headers)
console.log("verifierAuthRequest", result)

[14:02:27] verifierAuthRequest                                                                      ]8;id=908271;file:///tmp/ipykernel_14447/3364399777.py\3364399777.py]8;;\:]8;id=138749;file:///tmp/ipykernel_14447/3364399777.py#9\9]8;;\
           {                                                                                                       
               'client_id':                                                                                        
           'https://api-conformance.ebsi.eu/conformance/v2/verifier-mock/authentication-responses',                
               'response_type': 'id_token',                                                                        
               'scope': 'openid',                                                                                  
               'claims':                                                                                           
           '{"id_token":{"email":null},"vp_token":{"presentation_definition":{"id":"conformance_moc                
           k_vp_request","input_descriptors":[{"id":"conformance_mock_vp","name":"Conformance Mock                 
           VP","purpose":"Only accept a VP containing a Conformance Mock                                           
           VA","constraints":{"fields":[{"path":["$.vc.credentialSchema"],"filter":{"allOf":[{"type                
           ":"array","contains":{"type":"object","properties":{"id":{"type":"string","pattern":"htt                
           ps://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3kRpVjUFj4Bq8qHRENUHiZ                
           rVF5VgMBUe7biEafp1wf2J"}},"required":["id"]}}]}}]}}],"format":{"jwt_vp":{"alg":["ES256K"                
           ]}}}}}',                                                                                                
               'redirect_uri':                                                                                     
           'https://api-conformance.ebsi.eu/conformance/v2/verifier-mock/authentication-responses',                
               'nonce': '0a062816-fa2b-4a1a-a0fe-4c2ed5a5c1e1'                                                     
           }                                                                                                       

### Create Verifiable Presentation JWT

In [9]:
from ebsi_wallet.main import compute

encoded_credential = issue_031_credential
credential_issuer = credential['payload']['iss']

options = {
    "credential": encoded_credential,
    "client": client,
    "issuer": credential_issuer,
    "audience": credential_issuer
}

jwt_vp = await compute("createPresentationJwt", options=options)
console.log("createPresentationJwt", jwt_vp)

[14:02:38] createPresentationJwt                                                                   ]8;id=876892;file:///tmp/ipykernel_14447/2803278071.py\2803278071.py]8;;\:]8;id=106611;file:///tmp/ipykernel_14447/2803278071.py#14\14]8;;\
           {                                                                                                       
               'jwtVp':                                                                                            
           'eyJhbGciOiAiRVMyNTZLIiwgInR5cCI6ICJKV1QiLCAia2lkIjogImRpZDplYnNpOnprTmtWYlVoN29wMmVOMk                 
           hHbUZrTUN2aVRldVN1U2t5Z0tUWVBNS1NFdGFuQyNoSGpwbXk3enJpejBnSE9PRVZMT0g4ZE0wTnNMYXVleU5qR                 
           1N4S0dHWHNrIiwgImp3ayI6IHsia3R5IjogIkVDIiwgImNydiI6ICJzZWNwMjU2azEiLCAieCI6ICJMNHgybk5G                 
           UUFHWjktYzctWENfLWdoaENlaEdoR2hqYkpxTENiS2NhNWxVIiwgInkiOiAidFhBMjNmdWZSUHoyN1VwTkZsUUh                 
           Tc0Z2cTVuaDc2dWc1dGNBOWdmbHpjYyJ9fQ.eyJqdGkiOiAidXJuOmRpZDplZDk3NGY3Zi1iYzBjLTRkODUtYmM                 
           xMS1jMzA0ZTE1ODk0ZGEiLCAic3ViIjogImRpZDplYnNpOnprTmtWYlVoN29wMmVOMkhHbUZrTUN2aVRldVN1U2                 
           t5Z0tUWVBNS1NFdGFuQyIsICJpc3MiOiAiZGlkOmVic2k6emtOa1ZiVWg3b3AyZU4ySEdtRmtNQ3ZpVGV1U3VTa                 
           3lnS1RZUE1LU0V0YW5DIiwgIm5iZiI6IDE2NzMzMzk1NTEsICJleHAiOiAxNjczMzQwNDU4LCAiaWF0IjogMTY3                 
           MzMzOTU1OCwgImF1ZCI6ICJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkUzWiIsICJ2cCI6IHsiaWQiOiA                 
           idXJuOmRpZDplZDk3NGY3Zi1iYzBjLTRkODUtYmMxMS1jMzA0ZTE1ODk0ZGEiLCAiQGNvbnRleHQiOiBbImh0dH                 
           BzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0sICJ0eXBlIjogWyJWZXJpZmlhYmxlUHJlc2Vud                 
           GF0aW9uIl0sICJob2xkZXIiOiAiZGlkOmVic2k6emtOa1ZiVWg3b3AyZU4ySEdtRmtNQ3ZpVGV1U3VTa3lnS1RZ                 
           UE1LU0V0YW5DIiwgInZlcmlmaWFibGVDcmVkZW50aWFsIjogWyJleUpoYkdjaU9pSkZVekkxTmtzaUxDSjBlWEF                 
           pT2lKS1YxUWlMQ0pyYVdRaU9pSmthV1E2WldKemFUcDZZMlpqZDBkcVRFSnZhbU42ZHpsNWFHMVZSa1V6V2lOcl                 
           pYbHpMVEVpZlEuZXlKcWRHa2lPaUoxY200NlpHbGtPbUZoT0dNeFlXVmtMVGxtTkdVdE5HRTBPUzA0TldabExXV                 
           XhZalkwTkdaa01EZ3hZaUlzSW5OMVlpSTZJbVJwWkRwbFluTnBPbnByVG10V1lsVm9OMjl3TW1WT01raEhiVVpy                 
           VFVOMmFWUmxkVk4xVTJ0NVowdFVXVkJOUzFORmRHRnVReUlzSW1semN5STZJbVJwWkRwbFluTnBPbnBqWm1OM1I                 
           ycE1RbTlxWTNwM09YbG9iVlZHUlROYUlpd2libUptSWpveE5qY3pNek01TlRRMkxDSnBZWFFpT2pFMk56TXpNem                 
           sxTkRZc0luWmpJanA3SWtCamIyNTBaWGgwSWpwYkltaDBkSEJ6T2k4dmQzZDNMbmN6TG05eVp5OHlNREU0TDJOe                 
           VpXUmxiblJwWVd4ekwzWXhJbDBzSW1sa0lqb2lkWEp1T21ScFpEcGhZVGhqTVdGbFpDMDVaalJsTFRSaE5Ea3RP                 
           RFZtWlMxbE1XSTJORFJtWkRBNE1XSWlMQ0owZVhCbElqcGJJbFpsY21sbWFXRmliR1ZEY21Wa1pXNTBhV0ZzSWl                 
           3aVZtVnlhV1pwWVdKc1pVRjBkR1Z6ZEdGMGFXOXVJaXdpVm1WeWFXWnBZV0pzWlVsa0lsMHNJbWx6YzNWbGNpST                 
           ZJbVJwWkRwbFluTnBPbnBqWm1OM1IycE1RbTlxWTNwM09YbG9iVlZHUlROYUlpd2lhWE56ZFdGdVkyVkVZWFJsS                 
           WpvaU1qQXlNeTB3TVMweE1GUXdPRG96TWpveU5sb2lMQ0oyWVd4cFpFWnliMjBpT2lJeU1ESXpMVEF4TFRFd1ZE                 
           QTRPak15T2pJMldpSXNJbWx6YzNWbFpDSTZJakl3TWpNdE1ERXRNVEJVTURnNk16STZNalphSWl3aVkzSmxaR1Z                 
           1ZEdsaGJGTjFZbXBsWTNRaU9uc2lhV1FpT2lKa2FXUTZaV0p6YVRwNmEwNXJWbUpWYURkdmNESmxUakpJUjIxR2                 
           EwMURkbWxVWlhWVGRWTnJlV2RMVkZsUVRVdFRSWFJoYmtNaUxDSndaWEp6YjI1aGJFbGtaVzUwYVdacFpYSWlPa                 
           UpKVkM5RVJTOHhNak0wSWl3aVptRnRhV3g1VG1GdFpTSTZJa05oYzNSaFptbHZjbWtpTENKbWFYSnpkRTVoYldV                 
           aU9pSkNhV0Z1WTJFaUxDSmtZWFJsVDJaQ2FYSjBhQ0k2SWpFNU16QXRNVEF0TURFaWZTd2lZM0psWkdWdWRHbGh                 
           iRk5qYUdWdFlTSTZleUpwWkNJNkltaDBkSEJ6T2k4dllYQnBMV052Ym1admNtMWhibU5sTG1WaWMya3VaWFV2ZE                 
           hKMWMzUmxaQzF6WTJobGJXRnpMWEps

## VERIFY_031

In [11]:
options = {
    "credential": encoded_credential,
    "jwtVp": jwt_vp["jwtVp"],
    "client": client
}

vp_status = await conformance("verifierAuthResponse", options=options, headers={})
console.log("Verifiable presentation status", vp_status)

[14:02:45] Verifiable presentation status                                                           ]8;id=745452;file:///tmp/ipykernel_14447/3868489859.py\3868489859.py]8;;\:]8;id=714758;file:///tmp/ipykernel_14447/3868489859.py#8\8]8;;\
           {                                                                                                       
               'result': True,                                                                                     
               'validations': {                                                                                    
                   'vpFormat': {'status': True},                                                                   
                   'presentation': {'status': True},                                                               
                   'credential': {'status': True}                                                                  
               }                                                                                                   
           }                                                                                                       